In [1]:
# Collecting tweets from zenodo twitter dataset

# Download tweepy to use Twitter API
!pip3 install tweepy
import tweepy

You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Open file with Twitter developer keys
keys_file =open("TwitterAPI_Keys_Secret.txt", "r")
keys = keys_file.readlines()

# Twitter developer keys
consumer_key = str(keys[0])
consumer_secret = str(keys[1])
access_token = str(keys[2])
access_token_secret = str(keys[3])

consumer_key = consumer_key[:len(consumer_key)-1]
consumer_secret = consumer_secret[:len(consumer_secret)-1]
access_token = access_token[:len(access_token)-1]
access_token_secret = access_token_secret[:len(access_token_secret)-1]


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [4]:
import csv

tsv_file = "full_dataset-clean.tsv"
data = open(tsv_file)
read_tsv = csv.reader(data, delimiter="\t")

row_count = 1
tweet_ids = []
for row in read_tsv:
    if row_count <= 2:
        row_count += 1
        continue
    tweet_ids.append(row[0])
    #print(row_count, row[0])
    row_count += 1
    if row_count == 20:
        break

In [6]:
import re
!pip3 install unidecode
from unidecode import unidecode

# get full tweet text from tweet id
tweet = api.get_status(tweet_ids[6], tweet_mode="extended")
print(tweet.full_text + "\n")
#print(len(tweet.text))

# remove links beginning with http
text = re.sub(r"http\S+", "", tweet.full_text)
print(text + "\n")

# remove emojis
print(unidecode(text))

Defaulting to user installation because normal site-packages is not writeable
China - Media: WSJ says sources tell them mystery pneumonia is a new coronavirus - something that has been speculated for days.  https://t.co/ttsBah3olF h/t Shiloh, alert

China - Media: WSJ says sources tell them mystery pneumonia is a new coronavirus - something that has been speculated for days.   h/t Shiloh, alert

China - Media: WSJ says sources tell them mystery pneumonia is a new coronavirus - something that has been speculated for days.   h/t Shiloh, alert


You should consider upgrading via the 'c:\program files\python37\python.exe -m pip install --upgrade pip' command.
